# RFNSA STAD to 
# EMEG Equipment List, Prox5 RF Planning, CANRAD Validation



##### ©Haris Hassan | All rights reserved. You're not allowed to use it for commercial purposes unless given consent in written

if you're using google colab put your data in 
https://docs.google.com/spreadsheets/d/1H0gCKcFrVTpaCtJ3vFqDuzjTmIrvVptw2AJIgzERh88/


In [1]:
# Import Libraries
import sys
from IPython import get_ipython;
get_ipython().magic('reset -sf')
import pandas as pd
pd.set_option('display.max_colwidth', None)
from IPython.display import display, HTML
import re
import math
from itertools import count
from openpyxl.utils import get_column_letter
from openpyxl import Workbook
from openpyxl import load_workbook

In [2]:
# Replace with path to your excel file. 
input_file_path = r'C:\Users\Mewtwo\Downloads\New Microsoft Excel Worksheet (2).xlsx'

#input_file_path = r'RFNSA STAD.xlsx'
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    running_in_colab = True
    input_file_path =  False
    google_sheet_url = 'https://docs.google.com/spreadsheets/d/1H0gCKcFrVTpaCtJ3vFqDuzjTmIrvVptw2AJIgzERh88/'
else:
    google_sheet_url = False
    running_in_colab = False

In [3]:
class text_format:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def import_google_sheet(google_sheet_url):
  # 1. Authorizing google colab
    if running_in_colab:
        from google.colab import auth
        auth.authenticate_user()

        # 2. credentials for google sheets
        import gspread
        from google.auth import default
        creds, _ = default()

        # 3. authotizing the connection
        gc = gspread.authorize(creds)

        #4. Connecting 
        spreadsheet = gc.open_by_url(google_sheet_url)
        worksheet = spreadsheet.worksheet('Sheet1')

        #5. Exporting data to get_all_values gives a list of rows.
        rows = worksheet.get_all_values()

        # 6. Using pandas to convert to a DataFrame and render.
        df = pd.DataFrame(rows[1:], columns=rows[0])
    return df

def download_file_from_google_colab(output_file_path):
    from google.colab import files
    files.download(output_file_path)
    print(f"File downloaded: {output_file_path}")


# Function to convert from W to dBm
def watts_to_dBm(mW):
    return 10.*log10(mW) + 30

# Function to convert from dBm to W
def dBm_to_watts(dBm):
    return 10**((dBm-30)/10)

# Select 'Mount Height (m)' if available
def find_mount_height(data):   
    MountHeight = []
    for l, m in zip(data['Height - Phase Centre (m)'], data['Mount Height (m)']):
        if (m==0)|(m==''):
            MountHeight.append(l)
        else:
            MountHeight.append(m)
    return MountHeight

# Find technology carrier in antenna 
def find_carriers(df):
    Carriers = [''] * len(df)
    for idx in list( dict.fromkeys(df['Antenna Name ID Combined']) ):
        multicarrier = []
        for x in df.loc[df['Antenna Name ID Combined'] == idx, 'Operator']:
            multicarrier.append(x)       
        for index, operator in df.loc[df['Antenna Name ID Combined'] == idx, 'Operator'].iteritems():
            Carriers[index] = '/'.join(list( dict.fromkeys(multicarrier)))

    return Carriers

# Find technology in antenna 
def find_tech(df):
    tech = []
    for x, y in enumerate(df['Carrier']):
        if '/' in y:
            tech.append(''.join(df['System'][x].split(";")))
        else:
            tech.append(df['System'][x].split(";")[1].strip())
    return tech

# Find the sectors
def find_sectors(df):
    sectors = [''] * len(df)
    for idx in list( dict.fromkeys(df['Antenna Name ID Combined'])):
        Sectorstemp = []
        counter = 0
        for x in df.loc[df['Antenna Name ID Combined'] == idx, 'Sector']:
            Sectorstemp.append(str(x))
            counter +=1
            #display(idx)
        Sectorstemp.sort()
            
        for index, operator in df.loc[df['Antenna Name ID Combined'] == idx, 'Sector'].iteritems():
            sectors[index] = '/'.join(list( dict.fromkeys(Sectorstemp)))
    return sectors

def find_EDT(df):
    EDT = []
    for idx in list( dict.fromkeys(df['ID'],df['Antenna'])):
        EDTtemp = []
        counter = 0
        for x in df.loc[df['ID'] == idx, 'EDT']:
            EDTtemp.append(str(x))
            counter +=1
            #display(idx)
        EDTtemp.sort()
        for c in range(counter):
            EDT.append(', '.join(list( dict.fromkeys(EDTtemp))))
    return EDT

######## what should be the Assessment frequency? ########
def Find_assesment_frequency(TECH):
    Assess_Freq_list = []
    for t in TECH:
        try:
            i = re.search("(\d+)", str(t)).group(0)
            if any([x == i for x in ['900','850']]):
                Assess_Freq_list.append('900')
            elif any([i == 2100]):
                Assess_Freq_list.append('2100')
            elif i == '1800':
                Assess_Freq_list.append('1800')
            elif i== '2600':
                Assess_Freq_list.append('2600')
            elif any([x == i for x in ['700','750']]):
                Assess_Freq_list.append('750')
            elif i == '2300':
                Assess_Freq_list.append('2350')
            elif i == '3500':
                Assess_Freq_list.append('3500')
            else:
                Assess_Freq_list.append(i)
        except AttributeError:
            Assess_Freq_list.append(t.strip())
    return Assess_Freq_list

#Correcting microwavelinks
def fix_microwavelinks_power(df):
    templist = []
    for x in df['Powers']:
        if ';' not in str(x):
            templist.append(str(x)+';')
        else:
            templist.append(str(x))
    return templist

#Count the total ports of each antenna
def count_total_ports(df):
    total_ports = df['Powers'].str.count(';').fillna(0) + 1
    return total_ports

### Convert te Power(dBm) into Watts
def find_power_in_watts(df):
    temp_power = []
    for elm in df['Powers'].str.split(";"):
        if isinstance(elm, (list, tuple)):
            elm = [x for x in elm if x != '']  # Remove any empty strings from the list
            converted_values = []
            for sub in elm:
                if float(sub) >= 30:
                    try:
                        converted_values.append(round(dBm_to_watts(float(sub)),1))
                    except ValueError:
                        converted_values.append(sub)
                        pass  # Skip the current value if it cannot be converted to a float
                else:
                     try:
                        converted_values.append(round(dBm_to_watts(float(sub)),4))
                     except ValueError:
                        converted_values.append(sub)
                        pass  # Skip the current value if it cannot be converted to a float
            temp_power.append(converted_values)
        else:
            try:
                temp_power.append([dBm_to_watts(float(elm))])
            except ValueError:
                temp_power.append([])
    return temp_power

######## Which ports to add powers to? ########
def find_powered_ports(df):
    Mylist =[]

    for i in df['Powers']:
        count = 1
        lister = []
        if isinstance(i, str):
            for j in i:
                if j==';':
                    count+=1
                else:
                    lister.append(count)
            Mylist.append(lister)
        else:
            Mylist.append(list(str(i))) 

    new_k = []
    for elem in Mylist:
        kiterator=[]
        for el in elem:
            if el not in kiterator:
                kiterator.append(el)
        new_k.append(kiterator)
    k = new_k
    return k

#Add 0 before single digit
def fix_alphanumeric(input_list):
    fixed_list = []
    for x in input_list:
        templist = []
        for y in re.split(r'(\d+)', x.strip()):
            if re.match('\d{1,4}', y):
                y = y.zfill(5)
            templist.append(y)
        fixed_list.append(''.join(templist))
    return fixed_list

def generate_telstra_canrad(total_Ports_dict, df):
    listpow = []
    listfunc = []
    listport = []
    x = 0
    yprev = 0

    for AntId, AntPorts in total_Ports_dict.items():
        templistpow = [0] * int(AntPorts[0])
        templistfunc = [''] * int(AntPorts[0])
        templistport = list(range(1,int(AntPorts[0])+1))
        #tempfunc = [''] * int(AntPorts[0])
        tempdf = df.loc[(df['Antenna Name ID Combined']==AntId), ['Sector','Tech','WhereToAddPower','Powers (W)']]
        for sector, tech, port, power in zip(tempdf['Sector'], tempdf['Tech'], tempdf['WhereToAddPower'], 
                                             tempdf['Powers (W)']):
            for x, y in enumerate(port):       
                #if sector not in templistfunc[y]:
                templistfunc[y-1] += tech +'/'
                templistpow[y-1] = str(round(float(templistpow[y-1]) + float(power[x]),3))
        listpow.append(templistpow)
        listport.append(templistport)
        listfunc.append(templistfunc)
    
    listid_unwrap = []
    listport_unwrap = []
    listfunc_unwrap = []
    listpow_unwrap = []
    for a, z, x, y in zip([x for x in total_Ports_dict.keys()], listport, listfunc, listpow): 
        listid_unwrap.extend([a]*len(x))
        listport_unwrap.extend(z)
        listfunc_unwrap.extend(x)
        listpow_unwrap.extend(y)
    return listid_unwrap, listport_unwrap, listpow_unwrap, listfunc_unwrap

def find_tech_total_power(df):
    tech_total_power = []
    for powers in df['Powers (W)']:
        total_power = 0
        for power in powers:
            total_power = total_power + float(power)
        tech_total_power.append(str(total_power)+' W')
    return tech_total_power

    
def generate_equipment_list(total_Ports_dict, df):
    EDT_dict = {}
    for idantenna in list( dict.fromkeys(df['Antenna Name ID Combined'],df['Antenna'])):
        EDT_dict[idantenna] = list(dict.fromkeys(df.loc[df['Antenna Name ID Combined'] == idantenna, 'EDT']))
    
    ID_dict = {}
    for idantenna in list( dict.fromkeys(df['Antenna Name ID Combined'],df['Antenna'])):
        ID_dict[idantenna] = list(dict.fromkeys(df.loc[df['Antenna Name ID Combined'] == idantenna, 'ID']))
        
    listpow = []
    listsec = []
    x = 0
    yprev = 0
    for AntId, AntPorts in total_Ports_dict.items():
        templistpow = [''] * (int(AntPorts[0] / 2) + int(AntPorts[0] % 2))
        templistsec = [''] * (int(AntPorts[0] / 2) + int(AntPorts[0] % 2))
        tempdf = df.loc[(df['Antenna Name ID Combined']==AntId), ['EDT','System/Sector','WhereToAddPower','Powers (W)']]
        for sector, port, power in zip(tempdf['System/Sector'], tempdf['WhereToAddPower'], tempdf['Powers (W)']):
            display(AntId)
            if len(port)>1:
                for x, y in enumerate(port):       
                    if sector not in templistsec[math.ceil(int(y)/2)-1]:
                        templistsec[math.ceil(int(y)/2)-1] += sector +'\n'
                        templistpow[math.ceil(int(y)/2)-1] += '\n'
                    templistpow[math.ceil(int(y)/2)-1] += '+' + str(power[x])
            if len(port)==1:
                for x, y in enumerate(port):
                    display(math.ceil(int(y)/2)-1)
                    templistsec[math.ceil(int(y)/2)-1] += sector +'\n'
                    if y%2==0:
                        templistpow[math.ceil(int(y)/2)-1] += '\n0++' + str(power[x])
                    else:
                        templistpow[math.ceil(int(y)/2)-1] += '\n' + str(power[x]) + '++0'

        listpow.append(templistpow)
        listsec.append(templistsec)

    for index1, x in enumerate(listpow):
        for index2, y in enumerate(x):
            if y == '':
                listpow[index1][index2] = '0'

    for index1, x in enumerate(listsec):
        for index2, y in enumerate(x):
            if y == '':
                listsec[index1][index2] = '-'
    
    listid_unwrap = []
    listsec_unwrap = []
    listpow_unwrap = []
    listedt_unwrap = []
    for a, b, x, y, z in zip([x for x in total_Ports_dict.keys()], [x for x in ID_dict.values()], listsec, listpow, [x for x in EDT_dict.values()]): 
        listid_unwrap.extend(b)
        listid_unwrap.extend(['']*(len(x)-1))
        listsec_unwrap.extend(x)
        listpow_unwrap.extend(y)
        listedt_unwrap.extend(z*len(x))
    listpol = []
    for index, sys_sec in enumerate(listsec_unwrap):
        if 'link' in sys_sec or 'Radio' in sys_sec or 'Network' in sys_sec:
            listpol.append('')
        else:
            listpol.append('Dual Slant')
    
    return listid_unwrap, listedt_unwrap, listpol, listsec_unwrap, listpow_unwrap

def process_ibc_data(df_IBC):
    antenna_list = []
    structure_list = []
    system_list = []
    max_power_list = []
    for struc in list( dict.fromkeys(df_IBC['Structure']) ):
        structure_list.append(struc)
        antenna_list.append(list(dict.fromkeys(df_IBC.loc[df_IBC['Structure'] == struc, 'Antenna'])))
        system_list.append('\n'.join(x for x in dict.fromkeys(df_IBC.loc[(df_IBC['Structure'] == struc) & (df_IBC['System'].str.contains('IBC Composite')== False), 'System'].sort_values(ascending=True))))

    antenna_list = [[item for item in sublist if item not in ['MNC', 'DAS']] for sublist in antenna_list]
    system_list = [item.replace(' [IBC]', '').replace(' ;','') for item in system_list]

    for struc in antenna_list:
        max_power_list_temp = []
        for ant in struc:
            max_power = max(df_IBC.loc[df_IBC['Antenna'] == ant, 'Tech Total Power'])
            max_power_port =  ', '.join(str(x) for x in dict.fromkeys(df_IBC.loc[(df_IBC['Antenna'] == ant) & (df_IBC['Tech Total Power'] == max_power), 'ID']))
            max_power_list_temp.append(str(max_power) + '\n[' + str(max_power_port)+']')

        max_power_list.append(max_power_list_temp)

    antenna_list_unwrap = []
    structure_list_unwrap = []
    system_list_unwrap = []
    max_power_list_unwrap = []

    antenna_list_unwrap.extend(x for y in antenna_list for x in y)
    structure_list_unwrap.extend(l for x, y in zip(structure_list, antenna_list) for l in [str(x)]*len(y))
    system_list_unwrap.extend(l for x, y in zip(system_list, antenna_list) for l in [str(x)]*len(y))
    max_power_list_unwrap.extend(x for y in max_power_list for x in y)
    
    return structure_list_unwrap, antenna_list_unwrap, system_list_unwrap, max_power_list_unwrap

def export_to_excel(df, EquipmentList, CANRAD, dfSiteInfo, dfSiteNames, dfStructureInfo, IBC_data = False):
    nsa_number = str(dfSiteInfo['NSA'].values.tolist()[0]) 
    site_name = re.sub("[\\\\/]", " ", dfSiteNames['Site Name'].values.tolist()[0])
    output_file_name = nsa_number + ' ' + site_name + ' RFNSA Wrangled Data.xlsx'
    output_file_path = r'C:\Users\Mewtwo\Desktop'+'\\' + output_file_name
    if running_in_colab:
        output_file_path = output_file_name
    antenna_names_dict = {}
    for antenna in list( dict.fromkeys(df['Antenna']) ):
        antenna_names_dict[antenna] = list(dict.fromkeys(df.loc[df['Antenna'] == antenna, 'ID']))

    df3 = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in antenna_names_dict.items() ]))
    df3 = df3.fillna('')

    dfAntennaSettings = df[['ID','Bearing','Height','MDT','Total Ports','Carrier','Antenna','Sectors']].copy()
    dfAntennaSettings = dfAntennaSettings.drop_duplicates(keep='first')
    dfAntennaSettings = dfAntennaSettings.reset_index(drop=True)
    dfAntennaSettings.index += 1


    wb = Workbook()
    wb.save(output_file_path)
    
    #cell_format = wb.add_format({'bold': False, "font_name": "Arial"})

    writer = pd.ExcelWriter(output_file_path, engine='openpyxl')
    df3.to_excel(writer, sheet_name = 'Antenna IDs', index=False)
    df.to_excel(writer, sheet_name = 'Powers to Prox', columns = ['Antenna','ID',
                                                                      'Tech','Tech Total Power','WhereToAddPower',
                                                                      'Powers (W)', 'ProX5 ports', 
                                                                      'ProX5 powers','Assess Freq','EDT',
                                                                      'Notes'], index=False)
    dfAntennaSettings.to_excel(writer, sheet_name = 'Antenna Settings', index=True)
    pd.DataFrame([]).to_excel(writer, sheet_name = 'ProX5 Equip. Report', index=False)
    EquipmentList.set_index(EquipmentList.columns[:2].tolist()).to_excel(writer, sheet_name = 'EMEG List')
    #EquipmentList.set_index(EquipmentList.columns[:2].tolist()).to_excel('filename.xlsx')

    CANRAD.to_excel(writer, sheet_name = 'CANRAD', index=False)
    #df.to_excel(writer, sheet_name = 'df', index=False)
    dfSiteInfo.to_excel(writer, sheet_name = 'Site Info', index=False)
    dfSiteNames.to_excel(writer, sheet_name = 'Site Info', startcol = 3,index=False)
    dfStructureInfo.to_excel(writer, sheet_name = 'Site Info',startcol = 5, index=False)

    if not IBC_data.empty:
        IBC_data.to_excel(writer, sheet_name = 'IBC', index=False)
    # load the Excel file with openpyxl
    workbook = writer.book

    for sheet in workbook:
        for column in range(1, sheet.max_column + 2):
            letter = get_column_letter(column)
            #sheet.column_dimensions[letter].auto_size = True
            sheet.column_dimensions[letter].bestFit = True

    # save the modified workbook
    workbook.save(output_file_path)
    
    for sheetname in workbook.sheetnames:
        worksheet = workbook[sheetname]

        # set best fit attribute to true for all columns
        for column_cells in worksheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length + 5
            worksheet.column_dimensions[column_cells[0].column_letter].bestFit = True

    worksheet = workbook.active

    #export_to_html(EquipmentList,dfAntennaSettings,  df3, df, CANRAD)
   
    # set the width of column A to 15
    column_letter = get_column_letter(1)  # A
    column_dimension = worksheet.column_dimensions[column_letter]
    column_dimension.width = 15
    column_dimension.bestFit = True
    print(f"Results saved to {output_file_path}")
    if running_in_colab:
        download_file_from_google_colab(output_file_path)
        
def export_to_html(EquipmentList,dfAntennaSettings,  df3, df, CANRAD):
        EquipmentList.to_html(r'C:\Users\Mewtwo\Desktop\EMEGList.html', index=False)
        dfAntennaSettings.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata.html', index=True)
        df3.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata2.html',index=False)
        df.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata3.html', columns = ['Antenna','ID',
                                                                          'Tech','Tech Total Power','WhereToAddPower',
                                                                          'Powers (W)', 'ProX5 ports', 
                                                                          'ProX5 powers','Assess Freq','EDT',
                                                                          'Notes'], index=False)
        CANRAD.to_html(r'C:\Users\Mewtwo\Desktop\Canrad.html',index=False)

In [4]:
if __name__ == "__main__":
    try:
        if input_file_path:
            data = pd.read_excel(input_file_path)
    except NameError:
        print('No file with that name')
        input_file_path = False
        pass
    try:
        if google_sheet_url:
            data = import_google_sheet(google_sheet_url)
    except NameError:
        print('No sheet with that name')
        google_sheet_url = False
        pass
    data.fillna(value = 0,inplace = True)
    #remove the proposed Antennas
    data = data.drop(data[data['Existing/Proposed'] == 'Proposed'].index)
    data['Height'] = find_mount_height(data)

In [5]:
    dfSiteInfo = pd.DataFrame(data, columns=['NSA','Latitude', 'Longitude', 'Region'])
    dfSiteInfo['Coordinates'] = dfSiteInfo['Latitude'].astype(str) + ', ' + dfSiteInfo['Longitude'].astype(str)
    dfSiteInfo = dfSiteInfo.drop('Latitude', axis=1)
    dfSiteInfo = dfSiteInfo.drop('Longitude', axis=1)

    dfStructureInfo = pd.DataFrame(data, columns=['Structure Owner','Structure',
                                             'Site System (Structure No)',])
    dfSiteNames = pd.DataFrame(data, columns=['System','Site Name'])
    dfSiteNames['Operator'] = dfSiteNames['System'].str.split(" /").str[0]
    dfSiteNames = dfSiteNames.drop('System', axis=1)
    dfSiteNames['Site Name']=dfSiteNames['Site Name'].apply(lambda x: x.title())

    dfStructureInfo['Structure'] = ['Structure ' +f'{l}' + ': ' + ''.join(''.join(map(str, m))) 
                                    for l, m in zip(dfStructureInfo['Site System (Structure No)'],
                                                    dfStructureInfo['Structure'])]
    dfStructureInfo = dfStructureInfo.drop('Site System (Structure No)', axis=1)

    dfSiteNames = dfSiteNames.drop_duplicates(keep='first')
    dfSiteInfo = dfSiteInfo.drop_duplicates(keep='first')
    dfStructureInfo = dfStructureInfo.drop_duplicates(keep='first')

In [6]:
    ################ Create the dataframe from meaningful columns of STAD table and rename the columns ################
    df = pd.DataFrame(data, columns=['Antenna ID No','Add ID','Antenna Model','Sector','Height'
                                     ,'Bearing Degrees (true)','Site System (Structure No)',
                                     'Mech Downtilt','Elect Downtilt','System',
                                     'Port Number (Band Power per Port (dBm))','Band Power per Port (dBm)','Notes'])
    df = df.rename(columns={'Antenna Model': 'Antenna', 'Antenna ID No': 'ID', 'Bearing Degrees (true)': 'Bearing'
                      , 'Mech Downtilt': 'MDT', 'Elect Downtilt': 'EDT', 
                            'Site System (Structure No)':'Structure',
                       'Port Number (Band Power per Port (dBm))': 'Possible Ports', 
                       'Band Power per Port (dBm)': 'Powers'})
    df.fillna(value = 0,inplace = True)
    df = df.reset_index(drop=True)
    # Extracting Carrier (optus, telstra, Vodafone), technology/Frequency (i.e LTE900, NR2100, WCDMA850) information from System 
    df['System'] = df['System'].str.replace(' - LOCKED', '', regex=False)
    df['Antenna Name ID Combined'] =  df['Antenna'].astype(str) + df['ID'].astype(str)
    df['Configuration'] = df['System'].str.split("[").str[1]
    df['Configuration'].fillna(value = '',inplace = True)
    df['Configuration'] = df['Configuration'].str.replace(']', '', regex=False)
    df['System'] = df['System'].str.replace('[Macro]', '', regex=False)
    df['System'] = df['System'].str.replace('/', ';', 1, regex=False)
    df['Operator'] = df['System'].str.split(" ;").str[0]
    df['Carrier'] = find_carriers(df)
    df['Tech'] = find_tech(df)
    df['Assess Freq'] = Find_assesment_frequency(df['Tech'])

    df['Total Ports'] = count_total_ports(df)
    df['EDT'] = df['EDT'].astype(str).str.replace(' to ', '-', regex=False).replace(r'\(.*\)','', regex=True).replace('\.0', '', regex=True).str.strip()
    df['Sector'] = df['Sector'].astype(str).replace('\.0', '', regex=True)
    df['System/Sector'] = [''.join(''.join(map(str, m))+' - Sector ' +f'{l}') for l, m in zip(df['Sector'],df['Tech'])]
    df['Sectors'] = find_sectors(df)
    df['Powers'] = fix_microwavelinks_power(df)
    df['Elec. Tilt'] = find_EDT(df)
    df['Powers (W)'] = find_power_in_watts(df)
    df['WhereToAddPower'] = find_powered_ports(df)
    df['Tech Total Power'] = find_tech_total_power(df)
    df['ProX5 ports'] = [''] * len(df)
    df['ProX5 powers'] = [''] * len(df)
    
    df.loc[df['Tech'] == 'IBC Composite Systems', 'Configuration'] = 'IBC'
    
    if any([x == 'IBC' for x in df['Configuration']]):
        IBC_data =  pd.DataFrame([], columns=['Structure', 'Type/Make/Model','System/Function/Sector', 'Power (Watts)'])
        IBC_data['Structure'], IBC_data['Type/Make/Model'], IBC_data['System/Function/Sector'], IBC_data['Power (Watts)'] = process_ibc_data(df[df['Configuration']=='IBC'])
        #IBC_data['System/Function/Sector'] = IBC_data['System/Function/Sector'].str.replace(r"\['|',\s*'|'\]", '\n', regex=True)
    else:
        IBC_data=False
    df = df.drop(df[df['Configuration'] == 'IBC'].index)
    
    df["sorter"] =  fix_alphanumeric(df["Carrier"] + df['Antenna'].astype(str) + ['a']*len(df)+ df["ID"].astype(str)+ ['a']*len(df)+ df['Assess Freq'].astype(str) + ['a']*len(df)+ df["Tech Total Power"].astype(str))
    df.sort_values(by=['sorter'], inplace=True)
    df = df.reset_index(drop=True)
    df.index += 1
    
    total_ports_dict = {}
    for idantenna in list( dict.fromkeys(df['Antenna Name ID Combined'],df['Antenna'])):
        total_ports_dict[idantenna] = list(dict.fromkeys(df.loc[df['Antenna Name ID Combined'] == idantenna, 'Total Ports']))
    
    EquipmentList = pd.DataFrame([], columns=['ID','Elec. Tilt','Pol','System/Sector', 'Power (W)'])
    EquipmentList['ID'], EquipmentList['Elec. Tilt'], EquipmentList['Pol'],EquipmentList['System/Sector'], EquipmentList['Power (W)'] = generate_equipment_list(total_ports_dict, df)

    EquipmentList['Power (W)'] = EquipmentList['Power (W)'].str.replace('\+', '', 1, regex=True)
    EquipmentList['Power (W)'] = EquipmentList['Power (W)'].str.replace('\+\+', '+', regex=True)
    EquipmentList['Power (W)'] = EquipmentList['Power (W)'].str.replace('\\n\+', '\\n', regex=True)
    EquipmentList['Power (W)'] = EquipmentList['Power (W)'].str.replace('+', ' + ', regex=False)
    EquipmentList['Power (W)'] = EquipmentList['Power (W)'].str.strip()

    EquipmentList['System/Sector'] = EquipmentList['System/Sector'].str.replace('\n ', '\n', regex=False)
    EquipmentList['System/Sector'] = EquipmentList['System/Sector'].str.strip()
    CANRAD = pd.DataFrame([], columns=['ID','Port', 'Total Power', 'Function'])
    CANRAD['ID'], CANRAD['Port'], CANRAD['Total Power'], CANRAD['Function'] = generate_telstra_canrad(total_ports_dict, df)
    args = [df, EquipmentList, CANRAD, dfSiteInfo, dfSiteNames, dfStructureInfo]

    if not IBC_data.empty:
        args.append(IBC_data)

    export_to_excel(*args)

219
['Telstra ; LTE1800 [IBC]\nTelstra ; WCDMA850 [IBC]', 'Optus ; LTE1800 [IBC]\nOptus ; LTE2300 [IBC]\nOptus ; LTE2600 [IBC]\nOptus ; LTE700 [IBC]\nOptus ; LTE900 [IBC]\nOptus ; NR/LTE2100 [IBC]\nOptus ; NR2300 [IBC]\nOptus ; WCDMA900 [IBC]\nTelstra ; LTE1800 [IBC]\nTelstra ; LTE2600 [IBC]\nTelstra ; LTE700 [IBC]\nTelstra ; WCDMA850 [IBC]\nVodafone ; LTE1800 [IBC]\nVodafone ; LTE2100 [IBC]\nVodafone ; LTE850 [IBC]\nVodafone ; NR700 [IBC]\nVodafone ; WCDMA900 [IBC]', 'OVJV ; WCDMA2100 [IBC]']


'AIR3239 B4016-O'

0

'AIR3239 B4026-O'

0

...

'RR2VV-6533D-R631-V'

'RR2VV-6533D-R631-V'

'RR2VV-6533D-R631-V'

'RR2VV-6533D-R631-V'

Results saved to C:\Users\Mewtwo\Desktop\2190002 Greenacre Oleander St RFNSA Wrangled Data.xlsx
